# Movie Recommendation System: Popularity Model

**Background**:

An accurate, personalized recommendation system can improve business and sales and build customer satisfaction.


**Types of Recommendation Engines**:
1. **Popularity Model (see "popularity_movie_recommendation" notebook)**
    - Recommends the items which are liked by most number of users
    - Pros:
        - Fast
    - Cons:
        - No personalization bc popularity is defined by the user pool
    - Works because:
        - There is a division by section so user can look at the section
        - Making a good classier will become exponentially difficult as the number of users and items grow
        
2. **Recommendation algorithms**
    - **Content-based filtering (see "content_based_movie_recommendation" notebook)**
    - **Memory-based collaborative filtering (see "memory_based_collaborative_filtering_movie_recommendation" notebook)**
        1. User-user collaborative filtering
        2. Item-item collaborative filtering
    - **Model-based collaborative filtering**
        1. **Matrix Factorization**
            1. **Singular Vector Decomposition (see "matrix_factorization_svd_movie_recommendation" notebook)**
            2. Probabilitistic Matrix Factorization
            3. Non -ve Matrix Factorization
        2. **Deep learning/neural network (see "deep_learning_movie_recommendation" notebook)**
                
3. **Using a classifier to make recommendation**
    - Classifiers are parametric solutions that require some parameters of the user and item to be defined first
    - Pros:
        - Incorporates personalization
        - Works even if the user's past history is short or not available
    - Cons:
        - Features might not be availalbe or sufficient to create a good classifier
        - Making a good classifier will become exponentially difficult as the number of user and items grow
        
(https://medium.com/@james_aka_yale/the-4-recommendation-engines-that-can-predict-your-movie-tastes-bbec857b8223)

**Data**:

We will be using the online movie recommender service MovieLens' dataset collected from the MovieLens website. The datasets were collected over several periods of time.
Users were selected at random to be included in the data. All users have rated 20+ movies. No demographic information is included.

The data includes:
- 100K ratings (1-5) from 1000 users on 1700 movies
- Each user has rated 20+ movies
- Simple demographic information for the users, such as gender, age, occupation, zip, etc.
- Genre information of movies

(https://grouplens.org/datasets/movielens/10m/)

In [ ]:
!pip install numpy==1.10.4

In [1]:
import pandas as pd
import numpy as np
import scipy as sc
import pickle
import graphlab as gl

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

This non-commercial license of GraphLab Create for academic use is assigned to ngapui.leung@berkeley.edu and will expire on December 21, 2019.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1548535149.log


# Data

## Users Data

In [2]:
users = pd.read_pickle('users.pickle')
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


## Ratings Data

In [3]:
ratings = pd.read_pickle('ratings.pickle')
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


## Movies Data

In [4]:
movies = pd.read_pickle('movies.pickle')
movies.head()

,movie_id,movie_title,release_date,video_release_date,imdb_url,unknown,action,adventure,animation,childrens,comedy,crime,documentary,drama,fantasy,film_noir,horror,musical,mystery,romance,sci_fi,thriller,war,western,genres
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,animation|childrens|comedy
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,action|adventure|thriller
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,thriller
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,action|comedy|drama
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,crime|drama|thriller


## Train and Test Data

In [5]:
cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
train_data = pd.read_csv('ml-100k/ua.base', sep='\t', names=cols, encoding='latin-1')
test_data = pd.read_csv('ml-100k/ua.test', sep='\t', names=cols, encoding='latin-1')
print(train_data.shape, test_data.shape)

((90570, 4), (9430, 4))


In [6]:
train_data.to_pickle('train_data.pickle')
test_data.to_pickle('test_data.pickle')

# Popularity Movie Recommendation Model

(https://www.analyticsvidhya.com/blog/2016/06/quick-guide-build-recommendation-engine-python/)

**Since we are using GraphLab, we need to convert the data into SFrames.**

In [7]:
train_data_gl = gl.SFrame(train_data)
test_data_gl = gl.SFrame(test_data)

**Let's train a popularity model with Graphlab.**

In [21]:
pop_model = gl.popularity_recommender.create(train_data_gl, user_id='user_id', item_id='movie_id', target='rating')

Recsys training: model = popularity

Warning: Ignoring columns unix_timestamp;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 90570 observations with 943 users and 1680 items.

Data prepared in: 0.16482s

90570 observations to process; with 1680 unique items.

**Next, let's make the top 5 recommendations for the first 5 users.**

In [22]:
pop_rec = pop_model.recommend(users=range(1, 6), k=5)
pop_rec.print_rows(num_rows=25)

+---------+----------+-------+------+
| user_id | movie_id | score | rank |
+---------+----------+-------+------+
|    1    |   1467   |  5.0  |  1   |
|    1    |   1201   |  5.0  |  2   |
|    1    |   1189   |  5.0  |  3   |
|    1    |   1122   |  5.0  |  4   |
|    1    |   814    |  5.0  |  5   |
|    2    |   1467   |  5.0  |  1   |
|    2    |   1201   |  5.0  |  2   |
|    2    |   1189   |  5.0  |  3   |
|    2    |   1122   |  5.0  |  4   |
|    2    |   814    |  5.0  |  5   |
|    3    |   1467   |  5.0  |  1   |
|    3    |   1201   |  5.0  |  2   |
|    3    |   1189   |  5.0  |  3   |
|    3    |   1122   |  5.0  |  4   |
|    3    |   814    |  5.0  |  5   |
|    4    |   1467   |  5.0  |  1   |
|    4    |   1201   |  5.0  |  2   |
|    4    |   1189   |  5.0  |  3   |
|    4    |   1122   |  5.0  |  4   |
|    4    |   814    |  5.0  |  5   |
|    5    |   1467   |  5.0  |  1   |
|    5    |   1201   |  5.0  |  2   |
|    5    |   1189   |  5.0  |  3   |
|    5    | 

**As expected, the popularity model returns the same most popular movies for everyone.**

In [23]:
train_data.groupby(by='movie_id')['rating'].mean().sort_values(ascending=False).head(20)

movie_id
1500    5.000000
1293    5.000000
1122    5.000000
1189    5.000000
1656    5.000000
1201    5.000000
1599    5.000000
814     5.000000
1467    5.000000
1536    5.000000
1449    4.714286
1642    4.500000
1463    4.500000
1594    4.500000
1398    4.500000
114     4.491525
408     4.480769
169     4.476636
318     4.475836
483     4.459821
Name: rating, dtype: float64

# Evaluating Popularity Model

**If we recommended the top 10 most popular movies to every user, all of the first 10 movies in the list above would be recommended because they are the most popular with the highest rating of 5, which is the function of the popularity recommendation engine.**